# Using a BespokeFit-optimized force field

This is the second notebook in this workshop; start with `bespokefit.ipynb`. Make sure you run this notebook with the `openff-env` environment!

## Load the protein and ligand

In [ ]:
from openff.toolkit import Molecule, Topology

protein = Molecule.from_polymer_pdb("../pdb/protein.pdb")
ligand = Molecule.from_file("../sdf/lig_CAT-13a.sdf")
top = Topology.from_molecules([protein, ligand])

top.to_file("complex-bespoke.pdb")

## Prepare the force field

<div class="alert alert-block alert-warning">
<b>🚧 This code is not production-ready</b><br />
In this notebook, we use the SMIRNOFF port of the Amber ff14SB force field. This is currently the only mainstream protein force field that works with the OpenFF Toolkit, but it's slow and hasn't been rigorously checked against the original force field. The Open Force Field Initiative recommends waiting for OpenFF 3.0.0 "Rosemary", which will include protein parameters, before using this in production work.
</div>

In [ ]:
from openff.toolkit import ForceField

force_field = ForceField(
    "../offxml/openff-2.0.0_bespoke_cat13a.offxml",
    "ff14sb_off_impropers_0.0.3.offxml",
)

## Investigate parameters assigned to the ligand

In [ ]:
from pprint import pprint

labels = force_field.label_molecules(ligand.to_topology())

sage = ForceField("openff-2.0.0.offxml")
amber = ForceField("ff14sb_off_impropers_0.0.3.offxml")

params = []
for indices, parameter in labels[0]["ProperTorsions"].items():
    if sage["ProperTorsions"].get_parameter({"smirks": parameter.smirks}):
        source = "SAGE"
    elif amber["ProperTorsions"].get_parameter({"smirks": parameter.smirks}):
        source = "Amber"
    else:
        source = "BespokeFit"
    params.append((source, indices, {**parameter.to_dict()}))
pprint(params[:6], indent=2, width=-1)

## Box and solvate

In [ ]:
import openmm
import openmm.unit as openmm_unit
from pdbfixer import PDBFixer

fixer = PDBFixer("complex-bespoke.pdb")
fixer.addSolvent(
    padding=0.5 * openmm_unit.nanometer, ionicStrength=0.5 * openmm_unit.molar
)

with open("complex-bespoke_solvated.pdb", "w") as f:
    openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, f)

# Topology.from_openmm would also work here
top = Topology.from_pdb("complex-bespoke_solvated.pdb", unique_molecules=[ligand])

## Prepare OpenMM inputs

In [ ]:
omm_topology = top.to_openmm()
omm_system = force_field.create_openmm_system(top)

## Set up the OpenMM `Simulation`

In [ ]:
import openmm
import openmm.unit as omm_unit

# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * omm_unit.kelvin,
    1 / omm_unit.picosecond,
    0.002 * omm_unit.picoseconds,
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = openmm.app.Simulation(
    omm_topology,
    omm_system,
    integrator,
)
simulation.context.setPositions(top.get_positions().to_openmm())

# Add a reporter to record the structure every 100 steps (0.2 ps)
pdb_reporter = openmm.app.PDBReporter("trajectory-bespoke.pdb", 100)
simulation.reporters.append(pdb_reporter)

## Energy minimize

In [ ]:
import numpy as np

simulation.minimizeEnergy(
    tolerance=omm_unit.Quantity(value=50.0, unit=omm_unit.kilojoule_per_mole)
)
minimized_state = simulation.context.getState(
    getPositions=True, getEnergy=True, getForces=True
)

print(
    "Minimised to",
    minimized_state.getPotentialEnergy(),
    "with maximum force",
    max(
        np.sqrt(v.x * v.x + v.y * v.y + v.z * v.z) for v in minimized_state.getForces()
    ),
    minimized_state.getForces().unit.get_symbol(),
)

minimized_coords = minimized_state.getPositions()

## Simulate

In [ ]:
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())
simulation.runForClockTime(1.0 * omm_unit.minute)
print(
    f"Completed simulation time: {simulation.integrator.getStepSize() * simulation.currentStep}"
)

## Visualize

In [ ]:
from viz import visualize_protein_ligand

In [ ]:
visualize_protein_ligand("trajectory-bespoke.pdb", top)